In [2]:
from pathlib import Path 
import numpy as np
from PIL import Image

ls = []
# https://stackoverflow.com/questions/39909655/listing-of-all-files-in-directory 
files = [x for x in Path("./images_copy").rglob("*") if x.is_file()]# 

for file in files:
    if file.exists():
        img = np.array(Image.open(file).convert("L"))       
        path = str(file)
        number = path.split("\\")[1] 
        img = np.ravel(img)
        if number == "zero":
            x = np.append(img, [0])
        elif number == "one":
            x = np.append(img, [1])
        elif number == "two":    
            x = np.append(img, [2])
        elif number == "three":
            x = np.append(img, [3])
        elif number == "four":
            x = np.append(img, [4])
        elif number == "five":
            x = np.append(img, [5])
        elif number == "six":
            x = np.append(img, [6])
        elif number == "seven":
            x = np.append(img, [7])
        elif number == "eight":
            x = np.append(img, [8])
        elif number == "nine":
            x = np.append(img, [9])
        ls.append(x)
#         print(img.shape)

In [3]:
import pandas as pd 

df = pd.DataFrame(ls)

In [4]:
df = df.sample(frac=1).reset_index(drop=True)

df_2 = df[int(len(df)*0.8):]

In [6]:
import h2o 
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM Corretto-11.0.12.7.1 (build 11.0.12+7-LTS, mixed mode)
  Starting server from c:\Users\aravb\Desktop\PROGRAMS\H2O_GBM_TEST\.venv\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\aravb\AppData\Local\Temp\tmpsashuzlp
  JVM stdout: C:\Users\aravb\AppData\Local\Temp\tmpsashuzlp\h2o_abhardwaj_started_from_python.out
  JVM stderr: C:\Users\aravb\AppData\Local\Temp\tmpsashuzlp\h2o_abhardwaj_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.7
H2O_cluster_version_age:,27 days
H2O_cluster_name:,H2O_from_python_abhardwaj_7059a8
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.959 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"accepting new members, healthy"


In [7]:
data = h2o.import_file("https://h2o-public-test-data.s3.amazonaws.com/bigdata/laptop/mnist/train.csv.gz")

Parse progress: |██████████████████████████████████████████████████████████| 100%


In [8]:
hf = h2o.H2OFrame(df_2)
hf.col_names = data.col_names

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [9]:
train, test, valid = hf.split_frame(ratios=[.7, .15])

In [10]:
from h2o.estimators import H2OGradientBoostingEstimator

gbm = H2OGradientBoostingEstimator(ntrees = 784, max_depth = 3, learn_rate=0.01, stopping_metric="AUTO", stopping_rounds = 5, auc_type='MACRO_OVR')

In [11]:
response = "C785"
train[response] = train[response].asfactor()
predictors = train.columns
gbm.train(x=predictors, y=response, training_frame=train)

c:\Users\aravb\Desktop\PROGRAMS\H2O_GBM_TEST\.venv\lib\site-packages\h2o\estimators\estimator_base.py:201: RuntimeWarning: early stopping is enabled but neither score_tree_interval or score_each_iteration are defined. Early stopping will not be reproducible!
  warnings.warn(mesg["message"], RuntimeWarning)


gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [12]:
preds = gbm.predict(test)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [13]:
freqs = gbm.feature_frequencies(train)

In [14]:
preds.head

predict,p0,p1,p2,p3,p4,p5,p6,p7,p8,p9
7,0.00894978,0.00402865,0.0876796,0.00135066,0.00152253,0.0111632,0.00076619,0.881813,0.00108285,0.00164395
7,0.00830353,0.00101469,0.0360157,0.00126119,0.0120752,0.00379131,0.0021151,0.927071,0.00126123,0.00709119
9,0.0215321,0.0222897,0.0471797,0.0661275,0.364076,0.0329566,0.00700201,0.0154198,0.028755,0.394662
1,0.0184891,0.632119,0.0603454,0.0245129,0.0156329,0.00589893,0.0272784,0.0484582,0.105498,0.0617669
8,0.000511007,0.00185398,0.00474299,0.00904477,0.000930964,0.00607256,0.00128775,0.00196295,0.973033,0.000559733
9,0.017938,0.0141469,0.0509737,0.14117,0.12427,0.0126848,0.00625382,0.008375,0.0139489,0.610238
1,0.0012293,0.956766,0.0074557,0.00123005,0.00102296,0.000483904,0.00270917,0.0110289,0.000826876,0.0172475
2,0.00173952,0.0148365,0.76514,0.0331625,0.117057,0.00528955,0.00677598,0.0107955,0.0110361,0.0341675
9,0.00808847,0.0136544,0.0355238,0.00863027,0.00332236,0.00573219,0.00303215,0.0162789,0.00223542,0.903502
7,0.0147413,0.20237,0.110125,0.0546743,0.084278,0.0371064,0.0231994,0.312718,0.0104559,0.150332


<bound method H2OFrame.head of >

In [15]:
perf = gbm.model_performance(valid)

In [16]:
perf


ModelMetricsMultinomial: gbm
** Reported on test data. **

MSE: 0.10855062390018719
RMSE: 0.32947021701541884
LogLoss: 0.35604740952011965
Mean Per-Class Error: 0.0641024920325699
AUC: NaN
AUCPR: NaN
Multinomial auc values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).
Multinomial auc_pr values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,0,1,2,3,4,5,6,7,8,9,Error,Rate
0,119.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,3.0,0.048000,6 / 125
1,0.0,169.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.017442,3 / 172
2,1.0,0.0,127.0,0.0,0.0,5.0,2.0,7.0,0.0,1.0,0.111888,16 / 143
3,0.0,0.0,1.0,138.0,6.0,3.0,0.0,0.0,1.0,1.0,0.080000,12 / 150
4,0.0,0.0,0.0,0.0,163.0,0.0,0.0,1.0,0.0,8.0,0.052326,9 / 172
5,0.0,1.0,3.0,0.0,0.0,170.0,0.0,1.0,3.0,1.0,0.050279,9 / 179
6,1.0,2.0,4.0,0.0,0.0,1.0,154.0,0.0,0.0,1.0,0.055215,9 / 163
7,0.0,2.0,0.0,0.0,1.0,1.0,0.0,141.0,0.0,0.0,0.027586,4 / 145
8,0.0,0.0,0.0,5.0,0.0,2.0,2.0,0.0,167.0,3.0,0.067039,12 / 179
9,0.0,2.0,0.0,0.0,11.0,1.0,1.0,3.0,3.0,139.0,0.131250,21 / 160



Top-10 Hit Ratios: 


,k,hit_ratio
0,1,0.936398
1,2,0.977330
2,3,0.991184
3,4,0.996222
4,5,0.998111
5,6,0.998111
6,7,0.998741
7,8,1.000000
8,9,1.000000
9,10,1.000000
